In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [5]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
returns = dataset[0][0][0]
returns = (returns - returns.mean()) / returns.std()
X_train = returns.values
y_train = dataset[0][0][1].values

# returns = dataset[1][0][0]
# returns = (returns - returns.mean()) / returns.std()
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [7]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
y_train = get_one_hot(y_train, 2)
y_test = get_one_hot(y_test, 2)

In [8]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31, 2)
x test shape: (490, 31)
y test shape: (490, 31, 2)


In [9]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=510)

In [10]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=250)

In [11]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [12]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 240, 31)
y test shape: (250, 31, 2)


In [13]:
# Reshaping X_train for efficient modelling
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

In [14]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 240, 31)
y test shape: (250, 31, 2)


In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)

# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
# regressor.add(LSTM(units=25, return_sequences=True, input_shape=(240, 31)))
regressor.add(LSTM(units=10, input_shape=(240, 31), dropout=0.1))

# The output layer
# regressor.add(LSTM(25))
regressor.add(Dense(62, activation='relu'))
regressor.add(Reshape((31,2)))
# regressor.add(softmax(axis = -1))
regressor.add(Lambda(lambda  x: softmax(x, axis = -1)))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
# regressor.compile(loss='mean_squared_error',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
# Fitting to the training set
regressor.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=1000)
# regressor.fit(X_train,y_train,epochs=100,batch_size=10, validation_split=0.1, callbacks = [EarlyStopping(monitor='val_acc', patience=50),
#              ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])

Epoch 1/1000
1/1 [==============================] - 1s 847ms/step - loss: 0.6933 - acc: 0.5079
Epoch 2/1000
1/1 [==============================] - 0s 200ms/step - loss: 0.6933 - acc: 0.5096
Epoch 3/1000
1/1 [==============================] - 0s 199ms/step - loss: 0.6932 - acc: 0.5054
Epoch 4/1000
1/1 [==============================] - 0s 192ms/step - loss: 0.6928 - acc: 0.5124
Epoch 5/1000
1/1 [==============================] - 0s 196ms/step - loss: 0.6924 - acc: 0.5173
Epoch 6/1000
1/1 [==============================] - 0s 195ms/step - loss: 0.6923 - acc: 0.5130
Epoch 7/1000
1/1 [==============================] - 0s 200ms/step - loss: 0.6923 - acc: 0.5158
Epoch 8/1000
1/1 [==============================] - 0s 195ms/step - loss: 0.6917 - acc: 0.5186
Epoch 9/1000
1/1 [==============================] - 0s 206ms/step - loss: 0.6921 - acc: 0.5156
Epoch 10/1000
1/1 [==============================] - 0s 196ms/step - loss: 0.6920 - acc: 0.5186
Epoch 11/1000
1/1 [==============================

Epoch 87/1000
1/1 [==============================] - 0s 201ms/step - loss: 0.6848 - acc: 0.5493
Epoch 88/1000
1/1 [==============================] - 0s 203ms/step - loss: 0.6844 - acc: 0.5490
Epoch 89/1000
1/1 [==============================] - 0s 187ms/step - loss: 0.6845 - acc: 0.5508
Epoch 90/1000
1/1 [==============================] - 0s 189ms/step - loss: 0.6841 - acc: 0.5483
Epoch 91/1000
1/1 [==============================] - 0s 191ms/step - loss: 0.6839 - acc: 0.5517
Epoch 92/1000
1/1 [==============================] - 0s 200ms/step - loss: 0.6839 - acc: 0.5504
Epoch 93/1000
1/1 [==============================] - 0s 198ms/step - loss: 0.6842 - acc: 0.5513
Epoch 94/1000
1/1 [==============================] - 0s 195ms/step - loss: 0.6840 - acc: 0.5521
Epoch 95/1000
1/1 [==============================] - 0s 190ms/step - loss: 0.6843 - acc: 0.5499
Epoch 96/1000
1/1 [==============================] - 0s 188ms/step - loss: 0.6837 - acc: 0.5545
Epoch 97/1000
1/1 [=====================

Epoch 172/1000
1/1 [==============================] - 0s 200ms/step - loss: 0.6761 - acc: 0.5695
Epoch 173/1000
1/1 [==============================] - 0s 213ms/step - loss: 0.6760 - acc: 0.5712
Epoch 174/1000
1/1 [==============================] - 0s 199ms/step - loss: 0.6761 - acc: 0.5685
Epoch 175/1000
1/1 [==============================] - 0s 204ms/step - loss: 0.6757 - acc: 0.5731
Epoch 176/1000
1/1 [==============================] - 0s 201ms/step - loss: 0.6753 - acc: 0.5705
Epoch 177/1000
1/1 [==============================] - 0s 194ms/step - loss: 0.6751 - acc: 0.5706
Epoch 178/1000
1/1 [==============================] - 0s 200ms/step - loss: 0.6755 - acc: 0.5709
Epoch 179/1000
1/1 [==============================] - 0s 191ms/step - loss: 0.6754 - acc: 0.5706
Epoch 180/1000
1/1 [==============================] - 0s 203ms/step - loss: 0.6753 - acc: 0.5723
Epoch 181/1000
1/1 [==============================] - 0s 197ms/step - loss: 0.6760 - acc: 0.5659
Epoch 182/1000
1/1 [==========

Epoch 257/1000
1/1 [==============================] - 0s 199ms/step - loss: 0.6680 - acc: 0.5821
Epoch 258/1000
1/1 [==============================] - 0s 243ms/step - loss: 0.6678 - acc: 0.5821
Epoch 259/1000
1/1 [==============================] - 0s 207ms/step - loss: 0.6673 - acc: 0.5844
Epoch 260/1000
1/1 [==============================] - 0s 193ms/step - loss: 0.6678 - acc: 0.5828
Epoch 261/1000
1/1 [==============================] - 0s 201ms/step - loss: 0.6673 - acc: 0.5842
Epoch 262/1000
1/1 [==============================] - 0s 195ms/step - loss: 0.6675 - acc: 0.5843
Epoch 263/1000
1/1 [==============================] - 0s 201ms/step - loss: 0.6670 - acc: 0.5842
Epoch 264/1000
1/1 [==============================] - 0s 202ms/step - loss: 0.6673 - acc: 0.5846
Epoch 265/1000
1/1 [==============================] - 0s 195ms/step - loss: 0.6667 - acc: 0.5890
Epoch 266/1000
1/1 [==============================] - 0s 196ms/step - loss: 0.6670 - acc: 0.5823
Epoch 267/1000
1/1 [==========

Epoch 342/1000
1/1 [==============================] - 0s 203ms/step - loss: 0.6603 - acc: 0.5943
Epoch 343/1000
1/1 [==============================] - 0s 201ms/step - loss: 0.6596 - acc: 0.5962
Epoch 344/1000
1/1 [==============================] - 0s 203ms/step - loss: 0.6603 - acc: 0.5948
Epoch 345/1000
1/1 [==============================] - 0s 205ms/step - loss: 0.6598 - acc: 0.5947
Epoch 346/1000
1/1 [==============================] - 0s 210ms/step - loss: 0.6591 - acc: 0.5984
Epoch 347/1000
1/1 [==============================] - 0s 198ms/step - loss: 0.6598 - acc: 0.5952
Epoch 348/1000
1/1 [==============================] - 0s 197ms/step - loss: 0.6603 - acc: 0.5944
Epoch 349/1000
1/1 [==============================] - 0s 202ms/step - loss: 0.6595 - acc: 0.5956
Epoch 350/1000
1/1 [==============================] - 0s 200ms/step - loss: 0.6592 - acc: 0.5962
Epoch 351/1000
1/1 [==============================] - 0s 206ms/step - loss: 0.6589 - acc: 0.5984
Epoch 352/1000
1/1 [==========

Epoch 427/1000
1/1 [==============================] - 0s 205ms/step - loss: 0.6528 - acc: 0.6055
Epoch 428/1000
1/1 [==============================] - 0s 202ms/step - loss: 0.6519 - acc: 0.6083
Epoch 429/1000


In [ ]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [ ]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int

In [ ]:
(sum(y_test[:, :, 1] == label[:, :, 1])/(y_test.size/2)).sum()

In [ ]:
model_name = '../model/LSTM/my_model3.h5'
regressor.save(model_name)  # creates a HDF5 file 'my_model.h5'
del regressor  # deletes the existing model

In [ ]:
from keras.models import load_model
from keras.activations import softmax

# returns a compiled model
# identical to the previous one
regressor1 = load_model(model_name, custom_objects={"softmax": softmax})

In [ ]:
predicted_load = regressor1.predict(X_test)
label_load = predicted_load > 0.5
label_load = label_load * 1 # Convert boolean to int
(sum(y_test[:, :, 1] == label_load[:, :, 1])/(y_test.size/2)).sum()